# Zillow Property Search

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ruchita Dighe|
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 19 Jan 2026 | Ruchita D.| Create notebook. |

## Imports

In [ ]:
from google.colab import output, drive, files # specific to Google Colab
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import warnings

# settings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

**Functions**

In [ ]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

Locals & Constants

In [ ]:
# mount drive
drive.mount("/content/drive", force_remount=False)

# data location
file_dir = "/content/drive/My Drive/Colab Notebooks/"

Mounted at /content/drive


In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + "api_keys.csv")

# get keys
api_key = "3b12ffe0-346d-43ee-a703-e7db543d5ee6"
# df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key

Tampa city

In [ ]:
# zillow search url
listing_url = "https://www.zillow.com/tampa-fl/houses/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.96171153027345%2C%22east%22%3A-81.94684946972657%2C%22south%22%3A27.709897826482354%2C%22north%22%3A28.2786076346999%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A200000%2C%22max%22%3A550000%7D%2C%22mp%22%3A%7B%22min%22%3A946%2C%22max%22%3A2366%7D%2C%22beds%22%3A%7B%22min%22%3A2%7D%2C%22built%22%3A%7B%22min%22%3A1940%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"

# get listings
listing_response = get_listings(api_key, listing_url)

In [ ]:
# view all keys
listing_response.json().keys()

dict_keys(['is_success', 'data', 'message', 'info'])

In [ ]:
# check if request is successful
listing_response.json()["is_success"]

True

In [ ]:
# view count of properies returned in request
num_of_properties = listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 463


In [ ]:
# view all listings
df_listings = pd.json_normalize(listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_listings))
print("Number of columns:", len(df_listings.columns))
df_listings

Number of rows: 75
Number of columns: 85


,zpid,palsId,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,addressState,shouldShowRequestOnPrice,beds,baths,area,flexFieldText,contentType,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,info3String,brokerName,timeOnZillow,isPaidBuilderNewConstruction,latLong.latitude,latLong.longitude,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,hdpData.homeInfo.listing_sub_type.is_FSBO,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,hdpData.homeInfo.isRentalWithBasePrice,hdpData.homeInfo.listing_sub_type.is_forAuction,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.newConstructionType,builderName,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,hdpData.homeInfo.unit
0,45101135,24002_TB8463917,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/5002c2c05f0...,True,/homedetails/2906-W-Dewey-St-Tampa-FL-33607/45...,FOR_SALE,House for sale,"$389,900",$390K,"2906 W Dewey St, Tampa, FL 33607",FL,False,3,2.0,1248,Showcase,showcase,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,True,,False,False,https://photos.zillowstatic.com/fp/e39375731b4...,BHHS FLORIDA PROPERTIES GROUP,233870000,False,27.970865,-82.489460,45101135,2906 W Dewey St,33607,Tampa,FL,27.970865,-82.489460,389900.0,2.0,3.0,1248.0,SINGLE_FAMILY,FOR_SALE,2,False,False,386900.0,2625.0,True,False,False,FOR_SALE,389900.0,233870000,True,False,False,USD,USA,233518.0,5150.000000,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44962650,24002_TB8455541,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/0f7e5b7d3e0...,True,/homedetails/18101-Hamden-Park-Way-Tampa-FL-33...,FOR_SALE,House for sale,"$429,900",$430K,"18101 Hamden Park Way, Tampa, FL 33647",FL,False,3,2.0,1838,Showcase,showcase,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,True,,False,False,https://photos.zillowstatic.com/fp/e39375731b4...,KELLER WILLIAMS RLTY NEW TAMPA,480303000,False,28.137508,-82.332560,44962650,18101 Hamden Park Way,33647,Tampa,FL,28.137508,-82.332560,429900.0,2.0,3.0,1838.0,SINGLE_FAMILY,FOR_SALE,5,False,False,417600.0,2390.0,True,False,False,FOR_SALE,429900.0,480303000,True,False,False,USD,USA,364070.0,8395.000000,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,46380746,3741f92f8670c6fd80d015bd5e65db4e,ForSale,For Sale by Owner,https://photos.zillowstatic.com/fp/f45554b5f4a...,True,/homedetails/4727-Steel-Dust-Ln-Lutz-FL-33559/...,FOR_SALE,For sale by owner,"$390,000",$390K,"4727 Steel Dust Ln, Lutz, FL 33559",FL,False,3,2.0,1776,Peaceful views,homeInsight,False,False,ForSale,For Sale By Owner,False,False,False,False,False,False,False,,False,False,NaN,NaN,1606943000,False,28.155607,-82.399700,46380746,4727 Steel Dust Ln,33559,Lutz,FL,28.155607,-82.3

In [ ]:
save_path = file_dir + "tampa_listings.csv"
df_listings.to_csv(save_path, index=False)
print("File saved to:", save_path)


File saved to: /content/drive/My Drive/Colab Notebooks/tampa_listings.csv


### <font color="green">3. Property Details</font>

In [ ]:
# property unique id
zpid = "99616959"

# get property detail
prop_detail_response = get_property_detail(api_key, zpid)

In [ ]:
# view all keys
print(prop_detail_response.json().keys())

# check if request is successful
print("Request success:", prop_detail_response.json()["is_success"])

dict_keys(['is_success', 'data', 'message', 'info'])
Request success: True


In [ ]:
# view property details
df_prop = pd.json_normalize(prop_detail_response.json()['data'])
print("Number of rows:", len(df_prop))
print("Number of columns:", len(df_prop.columns))
df_prop

Number of rows: 1
Number of columns: 528


,zpid,palsId,buildingId,virtualTourUrl,isPremierBuilder,isShowcaseListing,homeStatus,richMedia,originalPhotos,bedrooms,bathrooms,contingentListingType,livingAreaValue,price,listingAccountUserId,isUndisclosedAddress,streetAddress,hdpUrl,tourViewCount,state,latitude,longitude,cityId,stateId,boroughId,countyId,homeType,responsivePhotos,hasApprovedThirdPartyVirtualTourUrl,isCurrentSignedInUserVerifiedOwner,listingAccount,description,primaryPublicVideo,richMediaVideos,isZillowOwned,hasPublicVideo,zestimate,isRecentStatusChange,isNonOwnerOccupied,brokerId,ssid,county,newConstructionType,daysOnZillow,livingArea,lotSize,lotAreaValue,yearBuilt,isFeatured,postingUrl,providerListingID,rentalApplicationsAcceptedType,brokerageName,currency,propertyTypeDimension,hdpTypeDimension,listingTypeDimension,featuredListingTypeDimension,brokerIdDimension,keystoneHomeStatus,pageUrlFragment,isRentalsLeadCapMet,isPaidMultiFamilyBrokerId,timeZone,photoCount,isListingClaimedByCurrentSignedInUser,isCurrentSignedInAgentResponsible,lastSoldPrice,livingAreaUnits,lotAreaUnits,isHousingConnector,responsivePhotosOriginalRatio,city,zipcode,livingAreaUnitsShort,streetViewTileImageUrlMediumLatLong,streetViewTileImageUrlMediumAddress,streetViewMetadataUrlMapLightboxAddress,thumb,neighborhoodMapThumb,interactiveFloorPlanUrl,lotPremium,listPriceLow,timeOnZillow,rentZestimate,zestimateLowPercent,zestimateHighPercent,schools,hasBadGeocode,streetViewImageUrl,countyFIPS,parcelId,taxHistory,priceHistory,country,rentZestimateURL,marketTrendsURL,listingProvider,isIncomeRestricted,whatILove,homeInsights,openHouseSchedule,desktopWebHdpImageLink,monthlyHoaFee,propertyTaxRate,hiResImageLink,mlsid,postingProductType,marketingName,listedBy,priceChange,priceChangeDate,priceChangeDateString,listingFeedID,comingSoonOnMarketDate,isPreforeclosureAuction,pals,hasPromotion,pageViewCount,favoriteCount,sellingSoon,promotionHeadline,promotionDescription,planNumber,specNumber,lotNumber,singleXXLPhoto,summaryFirstPhoto,moveInCompletionDate,moveInReady,propertyUpdatePageLink,moveHomeMapLocationLink,propertyEventLogLink,editPropertyHistorylink,hideZestimate,isRentalListingOffMarket,nearbyCities,nearbyNeighborhoods,nearbyZipcodes,abbreviatedAddress,building,boroughSearchUrl,communityUrl,restimateLowPercent,restimateHighPercent,homeValues,nearbyHomes,__typename,thirdPartyVirtualTour.providerKey,thirdPartyVirtualTour.externalUrl,thirdPartyVirtualTour.lightboxUrl,thirdPartyVirtualTour.staticUrl,thirdPartyVirtualTour.approved,listingMetadata.mustPreferMlsPhotos,listingMetadata.canShowOnMap,listingMetadata.FlexibleLayoutAB,listingMetadata.mustDisplayAttributionAboveLocalFacts,listingMetadata.mustAttributeOfficeNameBeforeAgentName,listingMetadata.mustDisplayAttributionListAgentEmail,listingMetadata.mustDisplayAttributionListAgentPhone,listingMetadata.mustDisplayAttributionListingOfficePhone,listingMetadata.mustDisplayDisclaimerBelowAttribution,listingMetadata.mustHighlightAgentName,listingMetadata.mustHighlightListOfficeName,listingMetadata.mustMakeListingAgentContactable,listingMetadata.FlexibleLayoutB,listingMetadata.FlexibleLayoutC,listingMetadata.FlexibleLayoutD,listingMetadata.FlexibleLayoutE,listingMetadata.FlexibleLayoutF,listingMetadata.FlexibleLayoutG,listingMetadata.FlexibleLayoutH,listingMetadata.FlexibleLayoutI,listingMetadata.FlexibleLayoutJ,listingMetadata.FlexibleLayoutK,listingMetadata.FlexibleLayoutL,listingMetadata.FlexibleLayoutM,listingMetadata.FlexibleLayoutN,listingMetadata.FlexibleLayoutO,listingMetadata.FlexibleLayoutP,listingMetadata.FlexibleLayoutQ,listingMetadata.FlexibleLayoutR,listingMetadata.FlexibleLayoutS,listingMetadata.FlexibleLayoutT,listingMetadata.FlexibleLayoutU,listingMetadata.FlexibleLayoutV,listingMetadata.FlexibleLayoutW,listingMetadata.FlexibleLayoutX,listingMetadata.FlexibleLayoutY,listingMetadata.FlexibleLayoutZ,listingMetadata.FlexibleLayoutAA,listingMetadata.passwordRequiredForZestimateMarketAnalysis,listingMetadata.canShowAutomatedValuationDisplay,listi

In [ ]:
print("Street address:", df_prop["streetAddress"].iloc[0])
print("City:", df_prop["city"].iloc[0])
print("State:", df_prop["state"].iloc[0])
print("Home status:", df_prop["homeStatus"].iloc[0])
print("Bedrooms:", df_prop["bedrooms"].iloc[0])
print("Bathrooms:", df_prop["bathrooms"].iloc[0])
print("Year built:", df_prop["yearBuilt"].iloc[0])
print("Zestimate: ${:,}".format(df_prop["zestimate"].iloc[0]))
print("Rent zestimate: ${:,}".format(df_prop["rentZestimate"].iloc[0]))
print("Description:")
df_prop["description"].iloc[0]

Street address: 3611 Potter St
City: Tampa
State: FL
Home status: SOLD
Bedrooms: 3
Bathrooms: 2
Year built: 2020
Zestimate: $371,700
Rent zestimate: $2,977
Description:


'WOW! What an amazing NEW BUILD with 3 bedrooms, 2 bathrooms, 1 car garage, fully fenced backyard and an incredible kitchen. Fall in love with the gorgeous landscaping in your front yard and a front porch with exterior lighting. As soon as you go inside immediately cherish the openness and TILED great room with your living room to your right, kitchen in the middle including GRANITE COUNTERTOPS, real wood cabinets, WALK-IN PANTRY with wire shelving racks and STAINLESS STEEL APPLIANCES. As you go farther inside, to your left you can find the TILED dining room with BLINDS and a new light fixture. Following the TILE FLOOR into the hallway on the right appreciate your washer and dryer in a closet, tucked away until it’s laundry day. As you keep going through the hallway you can find the CARPETED master bedroom with a WALK IN CLOSET, blinds and on the side is your TILED master bathroom with DUAL SINKS, granite countertops, real wood cabinets, and a stand up shower. As you exit the master bed

Tampa City All data

In [ ]:
def get_all_listings(api_key, base_url, max_pages=100):
    """
    Scrape Zillow listings across multiple pages, merging mapResults + listResults.
    """
    all_data = []
    page = 1

    while True:
        # Inject pagination into the searchQueryState
        paginated_url = base_url.replace(
            '%7B%22pagination%22%3A%7B%7D',
            f'%7B%22pagination%22%3A%7B%22currentPage%22%3A{page}%7D'
        )

        response = get_listings(api_key, paginated_url)
        json_data = response.json()

        # Safety check
        if not json_data.get("is_success", False):
            print(f"Page {page} failed.")
            break

        search_results = json_data["data"]["cat1"]["searchResults"]

        # Collect mapResults + listResults
        map_data = search_results.get("mapResults", [])
        list_data = search_results.get("listResults", [])

        if not map_data and not list_data:
            print(f"No more results at page {page}.")
            break

        df_map = pd.json_normalize(map_data) if map_data else pd.DataFrame()
        df_list = pd.json_normalize(list_data) if list_data else pd.DataFrame()

        df_page = pd.concat([df_map, df_list], ignore_index=True)
        all_data.append(df_page)

        print(f"Fetched page {page}, rows: {len(df_page)}")

        page += 1
        if page > max_pages:  # safety stop
            break

    # Combine all pages
    df_all = pd.concat(all_data, ignore_index=True)

    # Deduplicate by zpid
    if "zpid" in df_all.columns:
        df_all = df_all.drop_duplicates(subset=["zpid"])

    return df_all


After scraping Tampa city

In [ ]:
# Your Tampa search URL
listing_url = "https://www.zillow.com/tampa-fl/houses/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.96171153027345%2C%22east%22%3A-81.94684946972657%2C%22south%22%3A27.709897826482354%2C%22north%22%3A28.2786076346999%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A200000%2C%22max%22%3A550000%7D%2C%22mp%22%3A%7B%22min%22%3A946%2C%22max%22%3A2366%7D%2C%22beds%22%3A%7B%22min%22%3A2%7D%2C%22built%22%3A%7B%22min%22%3A1940%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"

df_all_listings = get_all_listings(api_key, listing_url)

print("Total properties collected:", len(df_all_listings))
df_all_listings


In [ ]:
save_path = file_dir + "tampa_all_listings.csv"
df_all_listings.to_csv(save_path, index=False)
print("File saved to:", save_path)


File saved to: /content/drive/My Drive/Colab Notebooks/tampa_all_listings.csv


**Alaska State**
---

In [ ]:
# Your Alaska STATE  search URL
listing_url = "https://www.zillow.com/ak/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A71.441059%2C%22south%22%3A51.175092%2C%22east%22%3A-129.979511%2C%22west%22%3A-179.231086%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A13%2C%22curatedCollection%22%3Anull%2C%22usersSearchTerm%22%3A%22alaska%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A3%2C%22regionType%22%3A2%7D%5D%2C%22category%22%3A%22cat1%22%7D"

df_all_listings = get_all_listings(api_key, listing_url)

print("Total properties collected:", len(df_all_listings))
df_all_listings

Fetched page 1, rows: 116
Fetched page 2, rows: 116
Fetched page 3, rows: 116
Fetched page 4, rows: 116
Fetched page 5, rows: 116
Fetched page 6, rows: 116
Fetched page 7, rows: 116
Fetched page 8, rows: 116
Fetched page 9, rows: 116
Fetched page 10, rows: 116
Fetched page 11, rows: 116
Fetched page 12, rows: 116
Fetched page 13, rows: 116
Fetched page 14, rows: 116
Fetched page 15, rows: 116
Fetched page 16, rows: 116
Fetched page 17, rows: 116
Fetched page 18, rows: 116
Fetched page 19, rows: 116
Fetched page 20, rows: 116
Fetched page 21, rows: 116
Fetched page 22, rows: 116
Fetched page 23, rows: 116
Fetched page 24, rows: 116
Fetched page 25, rows: 116
Fetched page 26, rows: 116
Fetched page 27, rows: 116
Fetched page 28, rows: 116
Fetched page 29, rows: 116
Fetched page 30, rows: 116
Fetched page 31, rows: 116
Fetched page 32, rows: 116
Fetched page 33, rows: 116
Fetched page 34, rows: 116
Fetched page 35, rows: 116
Fetched page 36, rows: 116
Fetched page 37, rows: 116
Fetched pa

,zpid,palsId,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,addressState,shouldShowRequestOnPrice,beds,baths,area,flexFieldText,contentType,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,info3String,brokerName,timeOnZillow,isPaidBuilderNewConstruction,latLong.latitude,latLong.longitude,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,hdpData.homeInfo.listing_sub_type.is_comingSoon,builderName,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.newConstructionType,hdpData.homeInfo.listing_sub_type.is_bankOwned,minBeds,minBaths,buildingId,isBuilding,canSaveBuilding,unitCount,listPriceIncludesRequiredMonthlyFees,lotAreaString,hdpData.homeInfo.unit,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,hdpData.homeInfo.listing_sub_type.is_FSBO,minArea,lotId,id,countryCurrency,unformattedPrice,addressStreet,addressCity,addressZipcode,isUndisclosedAddress,isZillowOwned,isSaved,zestimate,list,relaxed,carouselPhotosComposable.baseUrl,carouselPhotosComposable.communityBaseUrl,carouselPhotosComposable.photoData,carouselPhotosComposable.communityPhotoData,carouselPhotosComposable.isStaticUrls,availabilityDate,hasOpenHouse,openHouseStartDate,openHouseEndDate,openHouseDescription
0,43663,438001_26-353,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/470d88d4895...,True,/homedetails/11440-Doggie-Ave-Anchorage-AK-995...,FOR_SALE,House for sale,"$679,000",$679K,"11440 Doggie Ave, Anchorage, AK 99507",AK,False,3.0,4.0,2449.0,Showcase,showcase,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,True,,False,False,https://photos.zillowstatic.com/fp/d42b04ec66a...,"EXP Realty, LLC Anchorage",4.220470e+08,False,61.117077,-149.71790,43663.0,11440 Doggie Ave,99507,Anchorage,AK,61.117077,-149.71790,679000.0,4.0,3.0,2449.0,SINGLE_FAMILY,FOR_SALE,4.0,False,False,755100.0,3243.0,True,False,False,FOR_SALE,679000.0,4.220470e+08,True,False,False,USD,USA,607100.0,0.520000,acres,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39726,438001_26-336,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/c096ce3a482...,True,/homedetails/7341-Bulen-Dr-Anchorage-AK-99507/...,FOR_SALE,House for sale,"$475,000",$475K,"7341 Bulen Dr, Anchorage, AK 99507",AK,False,3.0,3.0,1624.0,Showcase,showcase,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,True,,False,False,https://photos.zillowstatic.com/fp/d42b04ec66a...,"Herrington and Company, LLC",4.832450e+08,False,61.153996,-149.82140,39726.0,7341 Bulen Dr,99507,Anchorage,AK,61.153996,-149.82140,475000.0,3.0,3.0,1624.0,SINGLE_FAMILY

In [ ]:
save_path = file_dir + "alaska_all_listings.csv"
df_all_listings.to_csv(save_path, index=False)
print("File saved to:", save_path)


File saved to: /content/drive/My Drive/Colab Notebooks/alaska_all_listings.csv


**Alabama State**
-

In [ ]:
# Your ALABAMA STATE  search URL
listing_url = "https://www.zillow.com/al/sold/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A36.06457290581162%2C%22south%22%3A29.015271489665583%2C%22east%22%3A-81.42377849218748%2C%22west%22%3A-91.93769450781248%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22mp%22%3A%7B%22min%22%3A946%2C%22max%22%3A2366%7D%2C%22built%22%3A%7B%22max%22%3Anull%7D%2C%22doz%22%3A%7B%22value%22%3A%2212m%22%7D%2C%22price%22%3A%7B%22min%22%3A0%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22usersSearchTerm%22%3A%22AL%22%2C%22mapZoom%22%3A7%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A4%2C%22regionType%22%3A2%7D%5D%2C%22pagination%22%3A%7B%7D%7D"

df_all_listings = get_all_listings(api_key, listing_url)

print("Total properties collected:", len(df_all_listings))
df_all_listings

Fetched page 1, rows: 116
Fetched page 2, rows: 116
Fetched page 3, rows: 116
Fetched page 4, rows: 116
Fetched page 5, rows: 116
Fetched page 6, rows: 116
Fetched page 7, rows: 116
Fetched page 8, rows: 116
Fetched page 9, rows: 116
Fetched page 10, rows: 116
Fetched page 11, rows: 116
Fetched page 12, rows: 116
Fetched page 13, rows: 116
Fetched page 14, rows: 116
Fetched page 15, rows: 116
Fetched page 16, rows: 116
Fetched page 17, rows: 116
Fetched page 18, rows: 116
Fetched page 19, rows: 116
Fetched page 20, rows: 116
Total properties collected: 75


,zpid,palsId,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,addressState,shouldShowRequestOnPrice,beds,baths,area,flexFieldText,contentType,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,timeOnZillow,isPaidBuilderNewConstruction,latLong.latitude,latLong.longitude,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.dateSold,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,info3String,brokerName,hdpData.homeInfo.zestimate,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceChange,info2String,info6String,hdpData.homeInfo.priceReduction,id,countryCurrency,soldPrice,unformattedPrice,addressStreet,addressCity,addressZipcode,isUndisclosedAddress,isZillowOwned,isSaved,list,relaxed,carouselPhotosComposable.baseUrl,carouselPhotosComposable.communityBaseUrl,carouselPhotosComposable.photoData,carouselPhotosComposable.communityPhotoData,carouselPhotosComposable.isStaticUrls,zestimate,providerListingId
0,109496581,583001_525097,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/e9e29116b63...,True,/homedetails/1153-County-Road-1605-Cullman-AL-...,SOLD,Sold,"$120,000",$120K,"1153 County Road 1605, Cullman, AL 35058",AL,False,3,2.0,1680.0,Sold 01/18/26,soldDate,False,False,RecentlySold,RecentChange,False,False,False,False,False,False,False,,False,False,126187000.0,False,34.191063,-86.76373,109496581,1153 County Road 1605,35058,Cullman,AL,34.191063,-86.76373,120000.0,1768723200000,2.0,3.0,1680.0,MANUFACTURED,RECENTLY_SOLD,1,False,False,1243,False,False,RECENTLY_SOLD,120000.0,126187000,True,False,False,USD,USA,88300.0,3.30,acres,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90299,194001_21423591,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/1d3804dd1e4...,True,/homedetails/7950-Joy-Rd-Blountsville-AL-35031...,SOLD,Sold,"$260,000",$260K,"7950 Joy Rd, Blountsville, AL 35031",AL,False,4,3.0,2292.0,Sold 01/16/26,soldDate,False,False,RecentlySold,Unknown Listed By,False,False,False,False,True,False,False,,False,False,298987000.0,False,34.061863,-86.59546,90299,7950 Joy Rd,35031,Blountsville,AL,34.061863,-86.59546,260000.0,1768550400000,3.0,4.0,2292.0,SINGLE_FAMILY,RECENTLY_SOLD,3,False,False,2990,False,False,RECENTLY_SOLD,260000.0,298987000,True,False,False,USD,USA,211900.0,21.10,acres,False,https://photos.zillowstatic.com/fp/4b0ddc61a23...,Keller Williams Realty Blount,275900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,166673,194001_21431401,RecentlySold,Non Owner Occupied,https://photos.zillowstatic.com/fp/b3749daedc9...,True,/homedetails/500-30th-St-N-Pell-City-AL-35125/...,SOLD,Sold,"$160,000",$160K,"500 30th St N, Pell City, AL 35125",AL,False,3,1.0,1333.0,Sold 01/16/26,soldDate,False,False,RecentlySold,Unknown Listed By,False,False,False,False,True,False,False,,False,False,298987000.0,False,33.593388,-86.27587,166673,500 30th St N,35125,Pell City,AL,33.593388,-86.

In [ ]:
save_path = file_dir + "alabama_all_listings.csv"
df_all_listings.to_csv(save_path, index=False)
print("File saved to:", save_path)


File saved to: /content/drive/My Drive/Colab Notebooks/alabama_all_listings.csv


**Washington State**
-

In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + "api_keys.csv")

# get keys
api_key = "40a048cd-0074-4b55-a99a-431d22cd54f0"
# df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key

In [ ]:
# Your washington STATE  search URL
listing_url = "https://www.zillow.com/washington-dc/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A39.0969935024422%2C%22south%22%3A38.68976210533418%2C%22east%22%3A-76.68601612451171%2C%22west%22%3A-77.34313587548827%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%2C%22curatedCollection%22%3Anull%2C%22usersSearchTerm%22%3A%22washington%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41568%2C%22regionType%22%3A6%7D%5D%2C%22category%22%3A%22cat1%22%7D"

df_all_listings = get_all_listings(api_key, listing_url)

print("Total properties collected:", len(df_all_listings))
df_all_listings

Fetched page 1, rows: 116
Fetched page 2, rows: 116
Fetched page 3, rows: 116
Fetched page 4, rows: 116
Fetched page 5, rows: 116
Fetched page 6, rows: 116
Fetched page 7, rows: 116
Fetched page 8, rows: 116
Fetched page 9, rows: 116
Fetched page 10, rows: 116
Fetched page 11, rows: 116
Fetched page 12, rows: 116
Fetched page 13, rows: 116
Fetched page 14, rows: 116
Fetched page 15, rows: 116
Fetched page 16, rows: 116
Fetched page 17, rows: 116
Fetched page 18, rows: 116
Fetched page 19, rows: 116
Fetched page 20, rows: 116
Fetched page 21, rows: 116
Fetched page 22, rows: 116
Fetched page 23, rows: 116
Fetched page 24, rows: 116
Fetched page 25, rows: 116
Fetched page 26, rows: 116
Fetched page 27, rows: 116
Fetched page 28, rows: 116
Fetched page 29, rows: 116
Fetched page 30, rows: 116
Fetched page 31, rows: 116
Fetched page 32, rows: 116
Fetched page 33, rows: 116
Fetched page 34, rows: 116
Fetched page 35, rows: 116
Fetched page 36, rows: 116
Fetched page 37, rows: 116
Fetched pa

,zpid,palsId,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,addressState,shouldShowRequestOnPrice,beds,baths,area,flexFieldText,contentType,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,info3String,brokerName,timeOnZillow,isPaidBuilderNewConstruction,latLong.latitude,latLong.longitude,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,hdpData.homeInfo.listing_sub_type.is_comingSoon,hdpData.homeInfo.comingSoonOnMarketDate,minBeds,minBaths,minArea,lotId,buildingName,buildingId,isBuilding,canSaveBuilding,unitCount,listPriceIncludesRequiredMonthlyFees,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceChange,hdpData.homeInfo.priceReduction,hdpData.homeInfo.unit,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.newConstructionType,hdpData.homeInfo.isRentalWithBasePrice,hdpData.homeInfo.listing_sub_type.is_forAuction,plid,communityName,builderName,isCdpResult,style,isComingSoonCommunity,id,countryCurrency,unformattedPrice,addressStreet,addressCity,addressZipcode,isUndisclosedAddress,isZillowOwned,isSaved,zestimate,list,relaxed,carouselPhotosComposable.baseUrl,carouselPhotosComposable.communityBaseUrl,carouselPhotosComposable.photoData,carouselPhotosComposable.communityPhotoData,carouselPhotosComposable.isStaticUrls,hasOpenHouse,openHouseStartDate,openHouseEndDate,openHouseDescription
0,472054,5632006_DCDC2241630,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/e584fbcabd9...,True,/homedetails/1328-Shepherd-St-NW-Washington-DC...,FOR_SALE,Townhouse for sale,"$1,150,000",$1.15M,"1328 Shepherd St NW, Washington, DC 20011",DC,False,4.0,4.0,2591.0,Showcase,showcase,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,True,,False,False,https://photos.zillowstatic.com/fp/3c0b241c63a...,Serhant,6.194960e+08,False,38.939530,-77.03098,4.720540e+05,1328 Shepherd St NW,20011,Washington,DC,38.939530,-77.03098,1150000.0,4.0,4.0,2591.0,TOWNHOUSE,FOR_SALE,7.0,False,False,1131500.0,4555.0,True,False,False,FOR_SALE,1150000.0,6.194960e+08,True,False,False,USD,USA,1162290.0,2928.0,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,433221,5632006_DCDC2213460,ComingSoon,Coming Soon,https://photos.zillowstatic.com/fp/054fe779a35...,True,/homedetails/3520-S-St-NW-Washington-DC-20007/...,FOR_SALE,Coming soon,"$1,400,000",$1.40M,"3520 S St NW, Washington, DC 20007",DC,False,5.0,4.0,1867.0,On Market Feb 19,comingSoonDate,False,False,ComingSoon,Coming Soon,False,True,False,False,True,False,True,,False,False,https://photos.zillowstatic.com/fp/3c0b241c63a...,Perennial Real Estate,1.487550e+10,False,38.914290,-7

In [ ]:
save_path = file_dir + "washington_all_listings.csv"
df_all_listings.to_csv(save_path, index=False)
print("File saved to:", save_path)


File saved to: /content/drive/My Drive/Colab Notebooks/washington_all_listings.csv


**California State**
-

In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + "api_keys.csv")

# get keys
api_key = "2b760b13-0cf5-4d2e-9ef8-828cb05a5055"
# df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key

# Your california STATE  search URL
listing_url = "https://www.zillow.com/ca/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A43.76785895415633%2C%22south%22%3A30.48269046035846%2C%22east%22%3A-108.79273298437501%2C%22west%22%3A-129.820565015625%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%2C%22curatedCollection%22%3Anull%2C%22category%22%3A%22cat1%22%2C%22usersSearchTerm%22%3A%22california%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%7D"

df_all_listings = get_all_listings(api_key, listing_url)

print("Total properties collected:", len(df_all_listings))
df_all_listings

Fetched page 1, rows: 116
Fetched page 2, rows: 116
Fetched page 3, rows: 116
Fetched page 4, rows: 116
Fetched page 5, rows: 116
Fetched page 6, rows: 116
Fetched page 7, rows: 116
Fetched page 8, rows: 116
Fetched page 9, rows: 116
Fetched page 10, rows: 116
Fetched page 11, rows: 116
Fetched page 12, rows: 116
Fetched page 13, rows: 116
Fetched page 14, rows: 116
Fetched page 15, rows: 116
Fetched page 16, rows: 116
Fetched page 17, rows: 116
Fetched page 18, rows: 116
Fetched page 19, rows: 116
Fetched page 20, rows: 116
Fetched page 21, rows: 116
Fetched page 22, rows: 116
Fetched page 23, rows: 116
Fetched page 24, rows: 116
Fetched page 25, rows: 116
Fetched page 26, rows: 116
Fetched page 27, rows: 116
Fetched page 28, rows: 116
Fetched page 29, rows: 116
Fetched page 30, rows: 116
Fetched page 31, rows: 116
Fetched page 32, rows: 116
Fetched page 33, rows: 116
Fetched page 34, rows: 116
Fetched page 35, rows: 116
Fetched page 36, rows: 116
Fetched page 37, rows: 116
Fetched pa

,zpid,palsId,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,addressState,shouldShowRequestOnPrice,beds,baths,area,flexFieldText,contentType,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,brokerName,timeOnZillow,isPaidBuilderNewConstruction,latLong.latitude,latLong.longitude,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,availabilityDate,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.newConstructionType,hdpData.homeInfo.unit,info6String,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,id,countryCurrency,unformattedPrice,addressStreet,addressCity,addressZipcode,isUndisclosedAddress,isZillowOwned,isSaved,zestimate,list,relaxed,carouselPhotosComposable.baseUrl,carouselPhotosComposable.communityBaseUrl,carouselPhotosComposable.photoData,carouselPhotosComposable.communityPhotoData,carouselPhotosComposable.isStaticUrls,hasOpenHouse,openHouseStartDate,openHouseEndDate,openHouseDescription
0,17356323,3009_IV26008232,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/2162ef27e24...,True,/homedetails/5443-Golondrina-Dr-San-Bernardino...,FOR_SALE,House for sale,"$539,900",$540K,"5443 Golondrina Dr, San Bernardino, CA 92404",CA,False,4,2.0,1698,Showcase,showcase,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,True,,False,False,ANOMALY REAL ESTATE,7.137330e+08,False,34.154995,-117.245930,17356323,5443 Golondrina Dr,92404,San Bernardino,CA,34.154995,-117.245930,539900.0,2.0,4.0,1698.0,SINGLE_FAMILY,FOR_SALE,8,False,False,546500.0,3619.0,True,False,False,FOR_SALE,539900.0,713733000,True,False,False,USD,USA,298058.0,7370.00,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63307185,11001_2601272,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/0ceace1c824...,True,/homedetails/5713-Canoe-Dr-Corona-CA-92880/633...,FOR_SALE,House for sale,"$949,000",$949K,"5713 Canoe Dr, Corona, CA 92880",CA,False,4,3.0,3604,Open: Sat 11am-2pm (1/24),openHouse,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,True,,False,False,"eXp Realty of California, Inc.",4.726810e+08,False,33.987583,-117.559370,63307185,5713 Canoe Dr,92880,Corona,CA,33.987583,-117.559370,949000.0,3.0,4.0,3604.0,SINGLE_FAMILY,FOR_SALE,5,False,False,945200.0,4724.0,True,False,False,FOR_SALE,949000.0,472681000,True,False,False,USD,USA,542125.0,0.29,acres,True,True,Sat. 11am-2pm,"[{'open_house_start': 1769281200000, 'open_hou...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66111107,17925003_11631792,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp

In [ ]:
save_path = file_dir + "california_all_listings.csv"
df_all_listings.to_csv(save_path, index=False)
print("File saved to:", save_path)


File saved to: /content/drive/My Drive/Colab Notebooks/california_all_listings.csv


**New York State**
-

In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + "api_keys.csv")

# get keys
api_key = "f1f8f47e-2eff-4386-8f9f-5911fc61e35d"
# df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key

# Your new york STATE  search URL
listing_url = "https://www.zillow.com/new-york-ny/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A41.09335646228189%2C%22south%22%3A40.29998455172489%2C%22east%22%3A-73.32256124902347%2C%22west%22%3A-74.63680075097659%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22curatedCollection%22%3Anull%2C%22usersSearchTerm%22%3A%22new%20york%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A6181%2C%22regionType%22%3A6%7D%5D%2C%22category%22%3A%22cat1%22%7D"

df_all_listings = get_all_listings(api_key, listing_url)

print("Total properties collected:", len(df_all_listings))
df_all_listings

Fetched page 1, rows: 116
Fetched page 2, rows: 116
Fetched page 3, rows: 116
Fetched page 4, rows: 116
Fetched page 5, rows: 116
Fetched page 6, rows: 116
Fetched page 7, rows: 116
Fetched page 8, rows: 116
Fetched page 9, rows: 116
Fetched page 10, rows: 116
Fetched page 11, rows: 116
Fetched page 12, rows: 116
Fetched page 13, rows: 116
Fetched page 14, rows: 116
Fetched page 15, rows: 116
Fetched page 16, rows: 116
Fetched page 17, rows: 116
Fetched page 18, rows: 116
Fetched page 19, rows: 116
Fetched page 20, rows: 116
Fetched page 21, rows: 116
Fetched page 22, rows: 116
Fetched page 23, rows: 116
Fetched page 24, rows: 116
Fetched page 25, rows: 116
Fetched page 26, rows: 116
Fetched page 27, rows: 116
Fetched page 28, rows: 116
Fetched page 29, rows: 116
Fetched page 30, rows: 116
Fetched page 31, rows: 116
Fetched page 32, rows: 116
Fetched page 33, rows: 116
Fetched page 34, rows: 116
Fetched page 35, rows: 116
Fetched page 36, rows: 116
Fetched page 37, rows: 116
Fetched pa

In [ ]:
save_path = file_dir + "NewYork_all_listings.csv"
df_all_listings.to_csv(save_path, index=False)
print("File saved to:", save_path)


File saved to: /content/drive/My Drive/Colab Notebooks/NewYork_all_listings.csv


**Texas state**
-

In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + "api_keys.csv")

# get keys
api_key = "4e0d83aa-c796-4f91-a1bf-9db2129b95cb"
# df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key

# Your new york STATE  search URL
listing_url = "https://www.zillow.com/tx/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A38.18965688121741%2C%22south%22%3A23.90973046413794%2C%22east%22%3A-89.562926484375%2C%22west%22%3A-110.590758515625%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22curatedCollection%22%3Anull%2C%22category%22%3A%22cat1%22%2C%22usersSearchTerm%22%3A%22texas%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A54%2C%22regionType%22%3A2%7D%5D%2C%22mapZoom%22%3A6%7D"

df_all_listings = get_all_listings(api_key, listing_url)

print("Total properties collected:", len(df_all_listings))
df_all_listings

Fetched page 1, rows: 116
Fetched page 2, rows: 116
Fetched page 3, rows: 116
Fetched page 4, rows: 116
Fetched page 5, rows: 116
Fetched page 6, rows: 116
Fetched page 7, rows: 116
Fetched page 8, rows: 116
Fetched page 9, rows: 116
Fetched page 10, rows: 116
Fetched page 11, rows: 116
Fetched page 12, rows: 116
Fetched page 13, rows: 116
Fetched page 14, rows: 116
Fetched page 15, rows: 116
Fetched page 16, rows: 116
Fetched page 17, rows: 116
Fetched page 18, rows: 116
Fetched page 19, rows: 116
Fetched page 20, rows: 116
Fetched page 21, rows: 116
Fetched page 22, rows: 116
Fetched page 23, rows: 116
Fetched page 24, rows: 116
Fetched page 25, rows: 116
Fetched page 26, rows: 116
Fetched page 27, rows: 116
Fetched page 28, rows: 116
Fetched page 29, rows: 116
Fetched page 30, rows: 116
Fetched page 31, rows: 116
Fetched page 32, rows: 116
Fetched page 33, rows: 116
Fetched page 34, rows: 116
Fetched page 35, rows: 116
Fetched page 36, rows: 116
Fetched page 37, rows: 116
Fetched pa

,zpid,palsId,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,addressState,shouldShowRequestOnPrice,beds,baths,area,flexFieldText,contentType,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,info3String,info6String,brokerName,timeOnZillow,isPaidBuilderNewConstruction,latLong.latitude,latLong.longitude,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,hdpData.homeInfo.zestimate,builderName,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,hdpData.homeInfo.newConstructionType,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,minBeds,minBaths,minArea,lotId,buildingId,isBuilding,canSaveBuilding,unitCount,listPriceIncludesRequiredMonthlyFees,hdpData.homeInfo.unit,availabilityDate,id,countryCurrency,unformattedPrice,addressStreet,addressCity,addressZipcode,isUndisclosedAddress,isZillowOwned,isSaved,list,relaxed,carouselPhotosComposable.baseUrl,carouselPhotosComposable.communityBaseUrl,carouselPhotosComposable.photoData,carouselPhotosComposable.communityPhotoData,carouselPhotosComposable.isStaticUrls,zestimate,hasOpenHouse,openHouseStartDate,openHouseEndDate,openHouseDescription
0,60080591,44006_1929370,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/1e44f9b99d5...,True,/homedetails/7254-Ruby-Palm-Pass-San-Antonio-T...,FOR_SALE,House for sale,"$255,000",$255K,"7254 Ruby Palm, San Antonio, TX 78218",TX,False,3.0,3.0,2096.0,Showcase,showcase,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,True,,False,False,https://photos.zillowstatic.com/fp/7e213a522d6...,Kenny Mota TREC #835687,Levi Rodgers Real Estate Group,2.967445e+09,False,29.499088,-98.38107,6.008059e+07,7254 Ruby Palm,78218,San Antonio,TX,29.499088,-98.38107,255000.0,3.0,3.0,2096.0,SINGLE_FAMILY,FOR_SALE,34.0,False,False,1857.0,True,False,False,FOR_SALE,255000.0,2.967445e+09,True,False,False,USD,USA,225960.0,6011.2800,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26643325,15004_21156357,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/cc10e7fc03a...,True,/homedetails/800-Clearlake-Dr-Allen-TX-75002/2...,FOR_SALE,House for sale,"$540,000",$540K,"800 Clearlake Dr, Allen, TX 75002",TX,False,5.0,4.0,3370.0,Showcase,showcase,False,False,ForSale,For Sale (Broker),False,True,False,False,False,False,True,,False,False,NaN,NaN,NaN,4.683860e+08,False,33.079266,-96.66051,2.664332e+07,800 Clearlake Dr,75002,Allen,TX,33.079266,-96.66051,540000.0,4.0,5.0,3370.0,SINGLE_FAMILY,FOR_SALE,5.0,False,False,4071.0,True,False,False,FOR_SALE,540000.0,4.683860e+08,True,False,False,USD,USA,535327.0,0.2600,acres,True,524500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
save_path = file_dir + "Texas_all_listings.csv"
df_all_listings.to_csv(save_path, index=False)
print("File saved to:", save_path)


File saved to: /content/drive/My Drive/Colab Notebooks/Texas_all_listings.csv


**Florida state**
-

In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + "api_keys.csv")

# get keys
api_key = "51286cc2-3aee-4daf-ab99-9629dac70826"
# df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key

# Your new york STATE  search URL
listing_url = "https://www.zillow.com/fl/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A31.388333381474002%2C%22south%22%3A23.983229926189743%2C%22east%22%3A-78.54764299218752%2C%22west%22%3A-89.06155900781252%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%7D%2C%22isListVisible%22%3Atrue%2C%22curatedCollection%22%3Anull%2C%22category%22%3A%22cat1%22%2C%22usersSearchTerm%22%3A%22florida%22%2C%22mapZoom%22%3A7%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A14%2C%22regionType%22%3A2%7D%5D%7D"

df_all_listings = get_all_listings(api_key, listing_url)

print("Total properties collected:", len(df_all_listings))
df_all_listings

Fetched page 1, rows: 116
Fetched page 2, rows: 116
Fetched page 3, rows: 116
Fetched page 4, rows: 116
Fetched page 5, rows: 116
Fetched page 6, rows: 116
Fetched page 7, rows: 116
Fetched page 8, rows: 116
Fetched page 9, rows: 116
Fetched page 10, rows: 116
Fetched page 11, rows: 116
Fetched page 12, rows: 116
Fetched page 13, rows: 116
Fetched page 14, rows: 116
Fetched page 15, rows: 116
Fetched page 16, rows: 116
Fetched page 17, rows: 116
Fetched page 18, rows: 116
Fetched page 19, rows: 116
Fetched page 20, rows: 116
Fetched page 21, rows: 116
Fetched page 22, rows: 116
Fetched page 23, rows: 116
Fetched page 24, rows: 116
Fetched page 25, rows: 116
Fetched page 26, rows: 116
Fetched page 27, rows: 116
Fetched page 28, rows: 116
Fetched page 29, rows: 116
Fetched page 30, rows: 116
Fetched page 31, rows: 116
Fetched page 32, rows: 116
Fetched page 33, rows: 116
Fetched page 34, rows: 116
Fetched page 35, rows: 116
Fetched page 36, rows: 116
Fetched page 37, rows: 116
Fetched pa

,zpid,palsId,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,addressState,shouldShowRequestOnPrice,beds,baths,area,flexFieldText,contentType,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,info3String,brokerName,timeOnZillow,isPaidBuilderNewConstruction,latLong.latitude,latLong.longitude,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.open_house_info.open_house_showing,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.zestimate,builderName,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.newConstructionType,info6String,hdpData.homeInfo.unit,id,countryCurrency,unformattedPrice,addressStreet,addressCity,addressZipcode,isUndisclosedAddress,isZillowOwned,isSaved,list,relaxed,hasOpenHouse,openHouseStartDate,openHouseEndDate,openHouseDescription,carouselPhotosComposable.baseUrl,carouselPhotosComposable.communityBaseUrl,carouselPhotosComposable.photoData,carouselPhotosComposable.communityPhotoData,carouselPhotosComposable.isStaticUrls,zestimate
0,47575768,24002_A4678402,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/e580ffde91f...,True,/homedetails/1471-Southland-Rd-Venice-FL-34293...,FOR_SALE,House for sale,"$370,000",$370K,"1471 Southland Rd, Venice, FL 34293",FL,False,3,2.0,1458,Open: Sun 1am-4pm (1/25),openHouse,False,False,ForSale,For Sale (Broker),False,True,False,False,True,False,True,,False,False,https://photos.zillowstatic.com/fp/e39375731b4...,WILLIAM RAVEIS REAL ESTATE,550787000.0,False,27.038013,-82.421180,47575768,1471 Southland Rd,34293,Venice,FL,27.038013,-82.421180,370000.0,2.0,3.0,1458.0,SINGLE_FAMILY,FOR_SALE,6,False,False,True,True,Sun. 1am-4pm,False,False,FOR_SALE,370000.0,"[{'open_house_start': 1769331600000, 'open_hou...",550787000,True,False,False,USD,USA,248400.0,8000.00,sqft,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44438264,22001_2125262,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/4e5aef3057c...,True,/homedetails/7942-Helston-Dr-Jacksonville-FL-3...,FOR_SALE,House for sale,"$209,900",$210K,"7942 HELSTON Drive, Jacksonville, FL 32208",FL,False,3,2.0,1170,Showcase,showcase,False,False,ForSale,For Sale (Broker),False,True,False,False,False,False,True,,False,False,NaN,NaN,574348000.0,False,30.392712,-81.696450,44438264,7942 HELSTON Drive,32208,Jacksonville,FL,30.392712,-81.696450,209900.0,2.0,3.0,1170.0,SINGLE_FAMILY,FOR_SALE,6,False,False,True,NaN,NaN,False,False,FOR_SALE,209900.0,NaN,574348000,True,False,False,USD,USA,115986.0,0.26,acres,True,1240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,337206304,22001_2124134,ForSale,For Sale by Agent,https://photos.zillowstatic.com/fp/d902f40f3c3...,True,/homedetails/114-Warren-Cir-Saint-Johns-FL-322...,FOR_SALE,House for sale,"$575

In [ ]:
save_path = file_dir + "Florida_all_listings.csv"
df_all_listings.to_csv(save_path, index=False)
print("File saved to:", save_path)


File saved to: /content/drive/My Drive/Colab Notebooks/Florida_all_listings.csv
